In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import gymnasium as gym
from d4rl.infos import REF_MIN_SCORE, REF_MAX_SCORE

from examples.offline.utils import load_buffer_d4rl
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py", line 594, in load_plugin_envs
    fn()
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shimmy/registration.py", line 262, in register_gymnasium_envs
    _register_dm_control_envs()
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shimmy/registration.py", line 26, in _register_dm_control_envs
    from shimmy.dm_control_compatibility import DmControlCompatibilityV0
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shim

In [2]:
task = "HalfCheetah-v3"
task_data = "halfcheetah-medium-v0"
device = "cuda:1"
learning_rate = 1e-3
batch_size = 1024
hidden_sizes = [512, 512, 512]

In [4]:
REF_MAX_SCORE["halfcheetah-medium-v0"]*66.4/100

8057.640000000001

In [40]:
IDbuffer = load_buffer_d4rl(task_data)

load datafile: 100%|██████████| 5/5 [00:00<00:00,  7.70it/s]


In [41]:
IDdata = np.concatenate((IDbuffer.obs, IDbuffer.act), axis=1)

In [42]:
# load numpy file
# ODdata = np.load("/data/user/R901105/dev/my_fork/tianshou/ODdata_halfcheetah_medium.npy")

In [43]:
import gym
env = gym.make(task)

In [44]:
env.action_space.high, env.action_space.low

(array([1., 1., 1., 1., 1., 1.], dtype=float32),
 array([-1., -1., -1., -1., -1., -1.], dtype=float32))

In [45]:
# generate num_repeat random actions for each observation in IDbuffer
# to use as a negative sample
num_repeat = 10
num_samples = num_repeat*len(IDbuffer)
rand_actions = np.random.uniform(low=env.action_space.low, high=env.action_space.high, size=(num_samples, env.action_space.shape[0])).astype(np.float32)
ODdata = np.concatenate((np.repeat(IDbuffer.obs, num_repeat, axis=0), rand_actions), axis=1)

In [46]:
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_sizes):
        super(MyModel, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.input_norm = nn.LayerNorm(hidden_sizes[0])
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_sizes[i], hidden_sizes[i+1])
            for i in range(len(hidden_sizes) - 1)
        ])
        self.hidden_norms = nn.ModuleList([
            nn.LayerNorm(hidden_sizes[i+1])
            for i in range(len(hidden_sizes) - 1)
        ])
        self.output_layer = nn.Linear(hidden_sizes[-1], 1)  # 1 output for energy score
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.input_norm(x)
        x = torch.relu(x)
        for hidden_layer, hidden_norm in zip(self.hidden_layers, self.hidden_norms):
            x = hidden_layer(x)
            x = hidden_norm(x)
            x = torch.relu(x)
        output = self.output_layer(x)
        return torch.sigmoid(output)

In [47]:
class MyData(Dataset):
    def __init__(self, ID, OD, device=device):
        self.ID = torch.from_numpy(ID)
        self.OD = torch.from_numpy(OD)
        self.X = torch.concatenate((self.ID, self.OD)).to(device)
        self.y = torch.concatenate((torch.ones(len(self.ID)), torch.zeros(len(self.OD)))).unsqueeze(-1).to(device)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [48]:
train_ID, test_ID = train_test_split(IDdata, test_size=0.2)
train_ID, val_ID = train_test_split(train_ID, test_size=0.2)
train_OD, test_OD = train_test_split(ODdata, test_size=0.2)
train_OD, val_OD = train_test_split(train_OD, test_size=0.2)

In [49]:
train_data = MyData(train_ID, train_OD)
val_data = MyData(val_ID, val_OD)
test_data = MyData(test_ID, test_OD)

In [50]:
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [51]:
train_features, train_labels = next(iter(train_dataloader))

In [52]:
input_size = list(train_features[0].shape)[0]

In [53]:
model = MyModel(input_size, hidden_sizes).to(device)
print(f"Model structure: {model}")

Model structure: MyModel(
  (input_layer): Linear(in_features=23, out_features=512, bias=True)
  (input_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (hidden_layers): ModuleList(
    (0-1): 2 x Linear(in_features=512, out_features=512, bias=True)
  )
  (hidden_norms): ModuleList(
    (0-1): 2 x LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (output_layer): Linear(in_features=512, out_features=1, bias=True)
)


In [92]:
loss_fn = nn.BCELoss()

In [93]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [94]:
from tqdm import tqdm
import time

def train_loop(dataloader, model, loss_fn, optimizer, val_dataloader=None, epochs=1):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        start_time = time.time()
        for X, y in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            optimizer.zero_grad()  # Zero gradients before forward pass
            pred = model(X)
            loss = loss_fn(pred, y)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(dataloader)
        end_time = time.time()
        elapsed_time = end_time - start_time

        if val_dataloader:
            model.eval()
            total_val_loss = 0
            with torch.no_grad():
                for X_val, y_val in val_dataloader:
                    pred_val = model(X_val)
                    val_loss = loss_fn(pred_val, y_val).item()
                    total_val_loss += val_loss

            avg_val_loss = total_val_loss / len(val_dataloader)
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Time: {elapsed_time:.2f}s")
    
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)

    return train_losses, val_losses

def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training and Validation Losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [95]:
train_losses, val_losses = train_loop(train_dataloader, model, loss_fn, optimizer, val_dataloader, epochs=200)

Epoch 1/200: 100%|██████████| 6871/6871 [01:15<00:00, 91.16it/s] 


Epoch 1/200, Train Loss: 0.1871, Validation Loss: 0.1267, Time: 75.38s


Epoch 2/200: 100%|██████████| 6871/6871 [01:14<00:00, 92.84it/s] 


Epoch 2/200, Train Loss: 0.1067, Validation Loss: 0.0911, Time: 74.01s


Epoch 3/200: 100%|██████████| 6871/6871 [01:17<00:00, 88.88it/s] 


...

Epoch 198/200, Train Loss: 0.0158, Validation Loss: 0.0194, Time: 78.52s


Epoch 199/200: 100%|██████████| 6871/6871 [01:17<00:00, 88.75it/s] 


Epoch 199/200, Train Loss: 0.0158, Validation Loss: 0.0198, Time: 77.42s


Epoch 200/200: 100%|██████████| 6871/6871 [01:16<00:00, 89.30it/s] 


Epoch 200/200, Train Loss: 0.0158, Validation Loss: 0.0204, Time: 76.94s


In [19]:
# plot_losses(train_losses, val_losses)

In [54]:
# load model
model.load_state_dict(torch.load("/data/user/R901105/dev/log/HalfCheetah-v3/action_classification/0/240504-201628/model.pt"))

<All keys matched successfully>

In [55]:
with torch.no_grad():
    test_pred = model(test_dataloader.dataset.X).round().cpu()
    test_labels = test_dataloader.dataset.y.cpu()

cm = confusion_matrix(test_labels, test_pred)

In [56]:
cm

array([[1958117,   39881],
       [   1467,  198333]])

In [57]:
cm / cm.sum(axis=1)[:, np.newaxis]

array([[0.98003952, 0.01996048],
       [0.00734234, 0.99265766]])

In [63]:
np.infty

inf

In [49]:
# save the model
# torch.save(model.state_dict(), "hopper_medium_sa.pt")